In [ ]:
import cv2
import mediapipe as mp
import math
from pynput.keyboard import Controller, Key

# Initialize mediapipe
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_draw = mp.solutions.drawing_utils

# Initialize keyboard controller
keyboard = Controller()

# Open webcam
cap = cv2.VideoCapture(2)  # Use the default camera (usually index 0)

finger_tip_ids = [4, 8, 12, 16, 20]  # Landmark IDs for the tips of five fingers

def calculate_distance(p1, p2):
    return math.sqrt((p2[0] - p1[0])**2 + (p2[1] - p1[1])**2)

def identify_fingers(hand_landmarks):
    finger_labels = []
    finger_positions = []
    for idx, landmark in enumerate(hand_landmarks.landmark):
        if idx in finger_tip_ids:
            finger_x = int(landmark.x * flipped_frame.shape[1])
            finger_y = int(landmark.y * flipped_frame.shape[0])
            finger_labels.append((finger_x, finger_y, f'Finger {finger_tip_ids.index(idx) + 1}'))
            finger_positions.append((finger_x, finger_y))
    return finger_labels, finger_positions

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        continue
    
    # Flip the frame horizontally
    flipped_frame = cv2.flip(frame, 1)
    
    # Convert BGR image to RGB
    rgb_frame = cv2.cvtColor(flipped_frame, cv2.COLOR_BGR2RGB)
    
    # Process the frame
    results = hands.process(rgb_frame)
    
    # Initialize hand states
    left_hand_positions = []
    right_hand_positions = []
    left_finger_labels = []  # Store finger labels for left hand
    right_finger_labels = []  # Store finger labels for right hand
    left_hand_label = "Left Hand"
    right_hand_label = "Right Hand"
    
    # Identify and handle fingers for each hand
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_draw.draw_landmarks(flipped_frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            
            # Identify and label fingers for one hand
            finger_labels, finger_positions = identify_fingers(hand_landmarks)
            
            # Calculate distance between finger 1 and 2 tips
            if len(finger_positions) >= 2:
                finger1_tip = finger_positions[0]
                finger2_tip = finger_positions[1]
                distance = calculate_distance(finger1_tip, finger2_tip)
                
                # Determine left or right hand based on finger 1 position
                finger1_x = finger_positions[0][0]
                if finger1_x < flipped_frame.shape[1] / 2:
                    left_hand_positions = finger_positions
                    left_finger_labels = finger_labels  # Store finger labels for left hand
                    left_hand_distance_label = f'{distance:.2f}'
                else:
                    right_hand_positions = finger_positions
                    right_finger_labels = finger_labels  # Store finger labels for right hand
                    right_hand_distance_label = f'{distance:.2f}'
        
    # Control actions based on hand positions
    if left_hand_positions:
        left_hand_distance = calculate_distance(left_hand_positions[0], left_hand_positions[1])
        if left_hand_distance < 80:
            keyboard.press(Key.left)
        else:
            keyboard.release(Key.left)
    
    if right_hand_positions:
        right_hand_distance = calculate_distance(right_hand_positions[0], right_hand_positions[1])
        if right_hand_distance < 80:
            keyboard.press(Key.right)
        else:
            keyboard.release(Key.right)
                
    # Draw finger tip information, lines, and hand label for both hands
    for label in left_finger_labels:
        cv2.putText(flipped_frame, label[2], (label[0], label[1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
    if left_hand_positions:
        cv2.putText(flipped_frame, left_hand_label, (left_hand_positions[0][0], left_hand_positions[0][1] - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
        cv2.putText(flipped_frame, left_hand_distance_label, ((left_hand_positions[0][0] + left_hand_positions[1][0]) // 2, (left_hand_positions[0][1] + left_hand_positions[1][1]) // 2), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1, cv2.LINE_AA)

    for label in right_finger_labels:
        cv2.putText(flipped_frame, label[2], (label[0], label[1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
    if right_hand_positions:
        cv2.putText(flipped_frame, right_hand_label, (right_hand_positions[0][0], right_hand_positions[0][1] - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
        cv2.putText(flipped_frame, right_hand_distance_label, ((right_hand_positions[0][0] + right_hand_positions[1][0]) // 2, (right_hand_positions[0][1] + right_hand_positions[1][1]) // 2), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1, cv2.LINE_AA)
    
    # Draw lines connecting fingers
    for i in range(0, len(left_hand_positions) - 1):
        cv2.line(flipped_frame, left_hand_positions[i], left_hand_positions[i + 1], (0, 255, 0), 2)
    for i in range(0, len(right_hand_positions) - 1):
        cv2.line(flipped_frame, right_hand_positions[i], right_hand_positions[i + 1], (0, 255, 0), 2)
    
    # Display the frame
    cv2.imshow('Hand Landmarks', flipped_frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()
